In [ ]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
#todo lastgang einfügen für lasten, entscheiden ob temperaturabhängig oder aufgeteilt 
#kosten definieren, stores, links 
#Gebäudetechnik

roof_area = 5500 #in qm
number_transporter = 78 # Anzahl E-Auto
capacitiy_battery_transporter = 40 # Batteriekapazität in kwh
inhouse_pv = 0.35 # Aufdach PV-Leistung maximal nach Haustechnik
investment_cost_Pv = x # in €
investment_cost_heat_pump = x # in € 
investment_cost_total = investment_cost_heat_pump + investment_cost_Pv # in €

verbrauch_autos = x # kwh/100km
annual_km = x # km  jährliche Fahrleistung pro Fahrzeug

#Verbräuche
building_load_hourly = roof_area*0.03 /8760 # in MW
DHW_load_hourly = roof_area*0.008 /8760 # in MW
Thermal_load_hourly = roof_area*0.05 /8760 # in MW
fleet_load_hourly = number_transporter * capacitiy_battery_transporter / 24 / 1000 # in MW

# Profile
renewable_profile_solar = pd.read_csv("ninja_pv_51.1601_13.csv", usecols=["electricity"]) # Lastprofil PV Inhouse
renewable_profile_solar = renewable_profile_solar["electricity"].astype(float)

network = pypsa.Network()
network.set_snapshots(range(8760))

#Knoten
network.add("Bus", name="electricity", carrier="electricity")
network.add("Bus", name="heat_0", carrier="heat")
network.add("Bus", name="heat_1", carrier="heat")

#Generator
network.add("Generator", name="PV", bus="electricity", carrier="electricity", 
            p_nom=inhouse_pv, 
            p_max_pu=renewable_profile_solar, 
            marginal_cost=5)
network.add("Generator", name="Grid", bus="electricity", carrier="electricity", 
            p_nom_expendable=True, 
            marginal_cost=300)

#Lasten
network.add("Load", name="Electric_Load", bus="electricity", p_set=building_load_hourly)
network.add("Load", name="Fleet_Load", bus="electricity", p_set=fleet_load_hourly) 
network.add("Load", name="DHW_Load", bus="heat_0", p_set=DHW_load_hourly) 
network.add("Load", name="Thermal_Load", bus="heat_1", p_set=Thermal_load_hourly) 

#Link
network.add("Link", name="Heat_Pump_0", bus0="electricity", bus1="heat_0", 
            efficiency=3.5, p_nom=10, marginal_cost=300) #Ökostrom
network.add("Link", name="Heat_Pump_1", bus0="electricity", bus1="heat_1", 
            efficiency=3.5, p_nom=10, marginal_cost=300)

#Wasserspeicher
network.add("Store", name="DHW_Storage", bus="heat_0", 
            carrier="heat", e_nom=20, e_cyclic=True, standing_loss=0.005)
network.add("Store", name="Thermal_Storage", bus="heat_1", 
            carrier="heat", e_nom=10, e_cyclic=True)


#Batteriespeicher des Gebäudes
network.add("Store", name="Building_Battery", bus="electricity", e_nom_extendable=True, e_nom_min=0, e_nom_max=10, capital_cost=300)  #e_nom in kWh & capital_cost in (€/kWh)


Index(['Thermal_Storage'], dtype='object')

In [26]:
network.optimize(solver_name = 'gurobi')

Index(['PV', 'Grid'], dtype='object', name='Generator')
Index(['DHW_Storage', 'Thermal_Storage'], dtype='object', name='Store')
Index(['electricity', 'heat_0', 'heat_1'], dtype='object', name='Bus')
Index(['Heat_Pump_0', 'Heat_Pump_1'], dtype='object', name='Link')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - threads: 1
 - method: 1
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 4/4 [00:00<00:00, 23.33it/s]
INFO:linopy.io: Writing time: 1.32s


Set parameter ServerTimeout to value 10


INFO:gurobipy:Set parameter ServerTimeout to value 10


Set parameter TokenServer to value "139.6.183.241"


INFO:gurobipy:Set parameter TokenServer to value "139.6.183.241"


...

Solved in 0 iterations and 0.05 seconds (0.02 work units)


INFO:gurobipy:Solved in 0 iterations and 0.05 seconds (0.02 work units)


Infeasible or unbounded model


INFO:gurobipy:Infeasible or unbounded model
INFO:linopy.solvers:Unable to save solution file. Raised error: Unable to retrieve attribute 'X'
Status: warning
Termination condition: infeasible_or_unbounded
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 4



('warning', 'infeasible_or_unbounded')

Optimale Leistung der Solaranlage: 0.0 MW
Optimale Netzbezug: 0.0 MW


KeyError: 'PV'

In [ ]:
# Heizung
Q_heat = network.links_t.p1["Heat_Pump_1"].sum() * dt
W_heat = network.links_t.p0["Heat_Pump_1"].sum() * dt
JAZ_heat = Q_heat / W_heat

# DHW
Q_dhw = network.links_t.p1["Heat_Pump_0"].sum() * dt
W_dhw = network.links_t.p0["Heat_Pump_0"].sum() * dt
JAZ_dhw = Q_dhw / W_dhw


In [ ]:
fleet_energy = network.loads_t.p["Fleet_Load"].sum() * dt


In [ ]:
print(f"PV-Ertrag: {pv_generation:.1f} MWh")
print(f"Autarkiegrad: {autarky*100:.1f} %")
print(f"JAZ Heizung: {JAZ_heat:.2f}")
print(f"JAZ DHW: {JAZ_dhw:.2f}")
print(f"E-Flotte Energiebedarf: {fleet_energy:.1f} MWh")
